# UnionML Demo

In [ ]:
from pictionary_app import model

num_classes = 200

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 1},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)

# 🚨🚨🚨 **NOTE FOR UNION.AI TEAM** 🚨🚨🚨 The following other settings will hopefully be cached and good
to go for use in the demo. You can try it out, and replace the code snippet above with one of the ones below.

### More labels:

- execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f27171bd39bec4f04929?duration=all

```python
num_classes = 200

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 1},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

### More labels, trained for longer

- execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f444bd69e68c747a59cf?duration=all

```python
num_classes = 200

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 20},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

### All labels, trained for longer

- cpu execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f23f637d2f3b74460a6f?duration=all
- gpu execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/f03bd4bc57adf4db1b46?duration=all

```python
num_classes = 345

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 20},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

### All labels, trained for longer much longer on gpu

- execution: https://playground.hosted.unionai.cloud/console/projects/unionml/domains/development/executions/fe652be0db07b473383e?duration=all

```python
num_classes = 345

execution = model.remote_train(
    hyperparameters={"num_classes": num_classes},
    trainer_kwargs={"num_epochs": 45},
    data_dir="./data",
    max_examples_per_class=10000,
    class_limit=num_classes,
)
```

In [ ]:
# wait for the execution to complete, then load model from remote
model.remote_load(execution)

In [ ]:
import gradio as gr

gr.Interface(
    fn=model.predict,
    inputs="sketchpad",
    outputs="label",
    live=True,
    allow_flagging="never",
).launch()

# Flytekit Demo

In [ ]:
from pictionary_app.dataset import QuickDrawDataset

dataset = QuickDrawDataset("/tmp/quickdraw_data", max_items_per_class=1000, class_limit=num_classes)
dataset

In [ ]:
model.artifact.model_object

Create a `FlyteRemote` object and fetch the batch prediction workflow

In [ ]:
from flytekit.remote import FlyteRemote
from flytekit.configuration import Config


remote = FlyteRemote(config=Config.auto(config_file="./config/config-remote.yaml"), data_upload_location="s3://open-compute-playground/data")
wf = remote.fetch_workflow(project="unionml", domain="development", name="flytekit_demo.batch_predictions.wf", version="v1")

Get features and labels

In [ ]:
feature_list = []
label_list = []
for i in range(5):
    X, y = dataset[i]
    feature_list.append(X.squeeze())
    label_list.append(y)

In [ ]:
# NOTE: make sure you have aws credentials for the s3://open-compute-playground bucket
execution = remote.execute(
    wf,
    inputs={"model_object": model.artifact.model_object, "feature_list": feature_list},
    project="unionml",
    domain="development",
    wait=True,
)

In [ ]:
execution